In [4]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import jsonlines
import pandas
import jieba
import re
import embedding
import numpy

In [2]:
#embedding.mergeDict() #生成字典，不需要执行

In [67]:
data_web = pandas.read_csv(r'D:\repositories\DaChuang\data\news_each_school\law_output.csv')

In [90]:
data_wx = pandas.read_csv(r'D:\repositories\DaChuang\data\公众号\ruc_caijing.csv')

In [46]:
data = pandas.concat([data_web,data_wx])

In [47]:
data

,datetime,source,url,title,content
0,2020-07-20,econ.ruc.edu.cn,http://econ.ruc.edu.cn/kxyj/xssx/b39bc18e9f474...,【光明日报】刘守英：中国农业的转型与现代化,新中国成立70多年来，伴随国家的现代化进程，农业现代化取得重大进展。当前，伴随着经济发展从高...
1,2020-07-20,econ.ruc.edu.cn,http://econ.ruc.edu.cn/kxyj/xssx/993a1cf7fc214...,【澎湃新闻】于泽：如何理解“两会”应对疫情的一揽子政策方案,刚刚闭幕的全国“两会”实质性地提出了我国应对新冠肺炎疫情的一揽子政策方案。当前还存在着对政策...
2,2020-07-20,econ.ruc.edu.cn,http://econ.ruc.edu.cn/kxyj/xssx/7c95ce6a2ebf4...,【中国证券报】刘元春：稳定经济基本盘，有效挖掘释放内需潜力,中国人民大学副校长刘元春近日接受中国证券报记者专访时表示，当前要以内部经济大循环为主体，快速...
3,2020-07-20,econ.ruc.edu.cn,http://econ.ruc.edu.cn/kxyj/xssx/36ea81c4c3ba4...,【中国宏观经济论坛】陈彦斌：稳增长过于依赖房地产会加剧经济分化,回顾过去二十多年我国宏观经济运行与房地产调控历程，一个很清晰的事实是每当经济下行压力较大时，...
4,2020-07-20,econ.ruc.edu.cn,http://econ.ruc.edu.cn/kxyj/xssx/87f35c67296d4...,【网易研究局】刘元春：美股走向与疫情基本面的背离越来越大,美国金融与疫情基本面的背离越来越大对于近期的剧烈波动，首先要明白，当下层面与基本面的背离越来...
...,...,...,...,...,...
744,2019-01-22,商苑经纬,http://mp.weixin.qq.com/s?__biz=MzA4MjE0OTcyMQ...,人在人大，寒假怎么“盘”？,终于“熬”过了“考试粥”，还要接着为考研实习奋战？图书馆啥时候开？什么时候能吃汇贤饺子？哪能...
745,2019-01-22,商苑经纬,http://mp.weixin.qq.com/s?__biz=MzA4MjE0OTcyMQ...,推广 |【微讲座】寒假系列-伯克利学姐解析金工申请与金融实习,讲座嘉宾C学姐清华经管本科，法律双学位在18申请季中获得加州大学伯克利分校金融工程硕士录取，...
746,2019-01-19,商苑经纬,http://mp.weixin.qq.com/s?__biz=MzA4MjE0OTcyMQ...,商国际 | 2019秋季院级交换项目系列分享会即将来袭,期待已久假期已经开始！与此同时，新一轮的国际交换项目申请也即将来袭！各项目院校都有哪些让人眼...
747,2019-01-11,商苑经纬,http://mp.weixin.qq.com/s?__biz=MzA4MjE0OTcyMQ...,关于启动中国人民大学商学院第十三届“管理之星”创业计划大赛暨第十五届学生课外学术论文大赛的通知,赛事简介中国人民大学商学院“管理之星”创业计划大赛暨学生课外学术论文大赛是由商学院团委主办的...


In [ ]:
#拼接
data = pandas.concat([data_web.正文,data_wx.content,data_web.标题,data_wx.title])

## 以下是训练word2vec的内容

In [ ]:
stop = embedding.getStopList()

In [ ]:
corpus = embedding.getCorpus() 获得语料

In [ ]:
jieba.load_userdict('../data/语料/dictionary.txt')

In [ ]:
corpus=[]
for sentence in data:
    #line = []
    segment = jieba.cut(sentence)
    seg = [word for word in list(segment) if word not in stop]
    
    #for word in simple_preprocess(seg,deacc=True,min_len=1):
    #    if word not in stop:
    #        line.append(word)
    corpus.append(seg)     

In [ ]:
with open(r'D:\repositories\DaChuang\data\teachers\teachers_information.json','r',encoding='utf-8') as f:
    for teacher in jsonlines.Reader(f):
        line = []
        line.append(teacher['department'])
        line.append(teacher['name'])
        try:
            line+=teacher['major']
        except:
            pass
        corpus.append(line)

In [ ]:
#输出语料
g = open(r'D:\codes\Pt_MarkDown\大创\data\corpus_information.txt','w',encoding='utf-8')
for sentence in corpus:
    line = ' '.join(sentence)
    g.write(line+'\n')
g.close()

In [ ]:
#统计关键词数量小于2的文档数
dictionary = embedding.getDict()
count_g = 0
docs = []
for doc in corpus:
    count = 0
    for word in doc:
        if word in dictionary:
            count+=1
    if count < 2:
        docs.append(doc)
        count_g += 1

In [ ]:
docs[2]

In [41]:
#训练模型
w2v_model = Word2Vec(corpus, size=100, min_count=1, workers=16)

In [42]:
w2v_model.save(r'D:\codes\Pt_MarkDown\大创\data\word_embeddings_information.model')

In [43]:
w2v_model.wv.similarity('窦志成','信息检索')

0.5212252

In [44]:
w2v_model.most_similar('信息学院')

[('我院', 0.6962107419967651),
 ('全院', 0.6765086650848389),
 ('秋冬季', 0.6276863813400269),
 ('座谈会', 0.6230578422546387),
 ('第六次', 0.6109772324562073),
 ('一个系列', 0.608765721321106),
 ('表示祝贺', 0.6076531410217285),
 ('数学系', 0.6065365076065063),
 ('人民大学', 0.5948217511177063),
 ('顺利', 0.5932497978210449)]

In [52]:
w2v_model.most_similar('微软')

[('亚洲', 0.920695960521698),
 ('研究院', 0.8535282611846924),
 ('工程院', 0.8212538957595825),
 ('JasonWu', 0.8192484974861145),
 ('InfoSys', 0.815918505191803),
 ('IBM', 0.7860386371612549),
 ('MSRA', 0.7722996473312378),
 ('QUIQ', 0.769305944442749),
 ('Pivotal', 0.7630451917648315),
 ('经理人', 0.7597249746322632)]

In [46]:
w2v_model.most_similar('深度学习')

[('知识图谱', 0.9517089128494263),
 ('隐私保护', 0.9451524019241333),
 ('自然语言处理', 0.9438310861587524),
 ('机器学习', 0.9416126012802124),
 ('信息网络', 0.9404984712600708),
 ('隐私', 0.93840491771698),
 ('高频', 0.9383750557899475),
 ('推荐系统', 0.9352732300758362),
 ('社会网络', 0.9343469738960266),
 ('挑战性', 0.9322288036346436)]

In [47]:
w2v_model.most_similar('宣讲')

[('交流平台', 0.756393551826477),
 ('调研', 0.7418028712272644),
 ('面对面', 0.7341629266738892),
 ('全程参加', 0.7308314442634583),
 ('开展调研', 0.7291984558105469),
 ('日常化', 0.7213379144668579),
 ('近期', 0.7122913002967834),
 ('互动', 0.708523690700531),
 ('形式', 0.7075490951538086),
 ('征集', 0.7058008909225464)]

In [48]:
w2v_model.most_similar('明德楼')

[('层', 0.9309592247009277),
 ('一层', 0.9287614822387695),
 ('1001', 0.9282403588294983),
 ('游泳馆', 0.9264535903930664),
 ('6308', 0.9256508946418762),
 ('大厦', 0.9253577589988708),
 ('北', 0.9230436086654663),
 ('主楼', 0.9199074506759644),
 ('二楼', 0.9188681840896606),
 ('汇贤', 0.9127611517906189)]

In [49]:
w2v_model.most_similar('SIGIR')

[('WWW', 0.9669920802116394),
 ('MSST', 0.9549739956855774),
 ('TKDE', 0.9537514448165894),
 ('CIKM', 0.9516314268112183),
 ('ICDE', 0.9500011205673218),
 ('EDBT', 0.9492573738098145),
 ('EI', 0.9484114646911621),
 ('AAAI', 0.9479264616966248),
 ('P', 0.9476830959320068),
 ('WSDM', 0.94678795337677)]